# Labelme2COCO

```
转换为 COCO 格式 label

python tools/dataset_converters/labelme2coco.py --img-dir demo/ \
                                                --labels-dir DSEC-Soft/test/ \
                                                --out DSEC-Soft/test_COCO_labels/test_COCO.json

```

## COCO format

In [13]:
# for example
ann_file_example = {
'images': [
    {
        'file_name': 'COCO_val2014_000000001268.jpg',
        'height': 427,
        'width': 640,
        'id': 1268
    },
    # ...
],

'annotations': [
    {
        'segmentation': [[192.81,
            247.09,
            # ...
            219.03,
            249.06]],  # 如果有 mask 标签且为多边形 XY 点坐标格式，则需要保证至少包括 3 个点坐标，否则为无效多边形
        'area': 1035.749,
        'iscrowd': 0,
        
        'image_id': 1268,
        # x1, y1, w, h
        'bbox': [192.81, 224.8, 74.73, 33.43],
        'category_id': 16,
        'id': 42986
    },
    # ...
],

'categories': [
    {'id': 0, 'name': 'car'},
 ]
}

## path config

In [14]:
%cd ~/mmyolo
from pathlib import Path

ann_base_path = Path('/home/jiangtianbin/event-rgb-fusion/DSEC_detection_labels')
ann_train_path = ann_base_path / Path('train_ann.json')
ann_test_path = ann_base_path / Path('test_ann.json')
ann_val_path = ann_base_path / Path('val_ann.json')

soft_ann_COCO_train_path = Path('/home/jiangtianbin/mmyolo/DSEC-Soft/soft_ann/soft_ann_COCO_train.json')
soft_ann_COCO_test_path = Path('/home/jiangtianbin/mmyolo/DSEC-Soft/soft_ann/soft_ann_COCO_test.json')

/home/jiangtianbin/mmyolo


## Set Info

In [15]:
view = 'LN'
# view = 'RN'

# gt_mode = 'train'
gt_mode = 'test'

if gt_mode == 'train':
    ann_path = ann_train_path
    out_path = Path(soft_ann_COCO_train_path)
else:
    ann_path = ann_test_path
    out_path = Path(soft_ann_COCO_test_path)

## load image pathes

In [16]:
import json

def read_josn(path):
    with open(path, 'r') as f:
        data = json.load(f)
        return data

ann_data = read_josn(ann_path)

image_path_list = []

for x in ann_data['images']:
    event_path = x['file_name']
    
    seq_name = event_path.split('/')[1]
    if view == 'LN':
        image_path = event_path.replace('events/', 'images/').replace('event_images/', 'left/rectified/').replace('.npy', '.png')
    elif view == 'RN':
        image_path = event_path.replace('events/', 'images/').replace('event_images/', 'right/rectified/').replace('.npy', '.png')
    else:
        assert 0

    # base = 'data/DSEC/train/'
    # image = plt.imread(base + image_path)
    # plt.imshow(image)
    
    image_path_list.append(image_path)

# for debug
# image_path_list = image_path_list[:2]

print(len(image_path_list))

2


## get exposure time

In [17]:
def get_exposure_time(image_path: Path):
    # image_path: must be a absolute path
    # /home/jiangtianbin/mmyolo/data/DSEC/train/images/zurich_city_04_a/images/left/rectified/000518.png
    # exposure time file_path:
    # /home/jiangtianbin/mmyolo/data/DSEC/train/images/zurich_city_04_a/images/left/exposure_timestamps.txt
    # return
    # exposure_start_timestamp_us, exposure_end_timestamp_us
    
    if isinstance(image_path, str):
        image_path = Path(image_path)

    if not image_path.is_absolute():
        assert 0

    image_id = image_path.stem
    image_index = int(image_id)

    exposure_file_path = image_path.parent.parent / Path('exposure_timestamps.txt')
    # print(exposure_file_path)

    with open(exposure_file_path, 'r') as f:
        lines = f.readlines()
        
        exposure_time = lines[image_index + 1].split(',')
        # print(exposure_time)
        return int(exposure_time[0]), int(exposure_time[1])

# print(    
# get_exposure_time(
#     Path('/home/jiangtianbin/mmyolo/data/DSEC/train/images/zurich_city_04_a/images/left/rectified/000518.png')
# )
# )

## to COCO

In [18]:
# changed from tools/dataset_converters/labelme2coco.py

import argparse
import json
from pathlib import Path
from typing import Optional

import numpy as np
from mmengine import track_iter_progress

from mmyolo.utils.misc import IMG_EXTENSIONS

def format_coco_annotations(points: list, image_id: int, annotations_id: int,
                            category_id: int) -> dict:
    """Gen COCO annotations format label from labelme format label.

    Labelme2COCO:
        points (list): Coordinates of four vertices of rectangle bbox.
        image_id (int): Image id.
        annotations_id (int): Annotations id.
        category_id (int): Image dir path.

    Return:
        annotation_info (dict): COCO annotation data.
    """
    annotation_info = dict()
    annotation_info['iscrowd'] = 0
    annotation_info['category_id'] = category_id
    annotation_info['id'] = annotations_id
    annotation_info['image_id'] = image_id

    # bbox is [x1, y1, w, h]
    annotation_info['bbox'] = [
        points[0][0], points[0][1], points[1][0] - points[0][0],
        points[1][1] - points[0][1]
    ]

    annotation_info['area'] = annotation_info['bbox'][2] * annotation_info[
        'bbox'][3]  # bbox w * h
    # segmentation_points = np.asarray(points).copy()
    # segmentation_points[1, :] = np.asarray(points)[2, :]
    # segmentation_points[2, :] = np.asarray(points)[1, :]
    # annotation_info['segmentation'] = [list(segmentation_points.flatten())]

    return annotation_info


def parse_labelme_to_coco(
        image_path_list: str,
        labels_root: str,
        all_classes_id: Optional[dict] = None) -> (dict, dict):
    """
    {
        "images": [
            {
                "height": 3000,
                "width": 4000,
                "id": 1,
                "file_name": "IMG_20210627_225110.jpg"
            },
            ...
        ],
        "categories": [
            {
                "id": 1,
                "name": "cat"
            },
            ...
        ],
        "annotations": [
            {
                "iscrowd": 0,
                "category_id": 1,
                "id": 1,
                "image_id": 1,
                "bbox": [
                    1183.7313232421875,
                    1230.0509033203125,
                    1270.9998779296875,
                    927.0848388671875
                ],
                "area": 1178324.7170306593,
                "segmentation": [
                    [
                        1183.7313232421875,
                        1230.0509033203125,
                        1183.7313232421875,
                        2157.1357421875,
                        2454.731201171875,
                        2157.1357421875,
                        2454.731201171875,
                        1230.0509033203125
                    ]
                ]
            },
            ...
        ]
    }
    """

    # init coco json field
    coco_json = {'images': [], 'categories': [], 'annotations': []}

    image_id = 0
    annotations_id = 0
    if all_classes_id is None:
        category_to_id = dict()
        categories_labels = []
    else:
        category_to_id = all_classes_id
        categories_labels = list(all_classes_id.keys())

        # add class_ids and class_names to the categories list in coco_json
        for class_name, class_id in category_to_id.items():
            coco_json['categories'].append({
                'id': class_id,
                'name': class_name
            })

    for img_file in track_iter_progress(image_path_list):

        # get label file according to the image file name
        seq_name = img_file.split('/')[1]
        label_path = Path(labels_root).joinpath(seq_name + '-' + Path(img_file).stem).with_suffix('.json')
        if not label_path.exists():
            print(f'Can not find label file: {label_path}, skip...')
            continue

        # load labelme label
        with open(label_path, encoding='utf-8') as f:
            labelme_data = json.load(f)

        image_id = image_id + 1  # coco id begin from 1

        # NOTE: need a relative path to DSRC-Soft
        file_path = labelme_data['imagePath']
        path_componet = file_path.split('/')
        seq_name = seq_name
        img_name = path_componet[-1]

        # this 'train/' is original DSEC dataset
        if view == 'LN':
            file_name = Path('train/images/') / Path(seq_name) / Path('images/left/rectified') / Path(img_name)
        elif view == 'RN':
            file_name = Path('train/images/') / Path(seq_name) / Path('images/right/rectified') / Path(img_name)
        else:
            assert 0

        exposure_start_timestamp_us, exposure_end_timestamp_us = get_exposure_time(file_path)

        # update coco 'images' field
        coco_json['images'].append({
            'height':
            labelme_data['imageHeight'],
            'width':
            labelme_data['imageWidth'],
            'id':
            image_id,
            'file_name':
            str(file_name),
            'exposure_start_timestamp_us':
            exposure_start_timestamp_us,
            'exposure_end_timestamp_us':
            exposure_end_timestamp_us
        })

        # NOTE: a shape is a object
        for label_shapes in labelme_data['shapes']:

            # Update coco 'categories' field
            class_name = label_shapes['label']

            # filter the categories
            # posible class_name = ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck']
            # final_class_names = ['car', 'large_vehicle' 'person'] 0 1 2

            # -1 means to filter it
            categories_filter_id = {
                'person': 2,
                'bicycle': -1,
                'car': 0,
                'motorcycle': -1,
                'bus': 1,
                'truck': 1
            }
            if categories_filter_id[class_name] == -1:
                continue
            if class_name == 'bus' or class_name == 'truck':
                class_name = 'large_vehicle'


            if (all_classes_id is None) and (class_name
                                             not in categories_labels):
                # only update when not been added before
                coco_json['categories'].append({
                    'id':
                    len(categories_labels) + 1,  # categories id start with 1
                    'name': class_name
                })
                categories_labels.append(class_name)
                category_to_id[class_name] = len(categories_labels)

            elif (all_classes_id is not None) and (class_name
                                                   not in categories_labels):
                # check class name
                raise ValueError(f'Got unexpected class name {class_name}, '
                                 'which is not in your `--class-id-txt`.')

            # get shape type and convert it to coco format
            shape_type = label_shapes['shape_type']
            if shape_type != 'rectangle':
                print(f'not support `{shape_type}` yet, skip...')
                continue

            annotations_id = annotations_id + 1
            # convert point from [xmin, ymin, xmax, ymax] to [x1, y1, w, h]
            (x1, y1), (x2, y2) = label_shapes['points']
            x1, x2 = sorted([x1, x2])  # xmin, xmax
            y1, y2 = sorted([y1, y2])  # ymin, ymax
            points = [[x1, y1], [x2, y2], [x1, y2], [x2, y1]]
            coco_annotations = format_coco_annotations(
                points, image_id, annotations_id, category_to_id[class_name])
            coco_json['annotations'].append(coco_annotations)
    
    print(f'Total image = {image_id}')
    print(f'Total annotations = {annotations_id}')
    print(f'Number of categories = {len(categories_labels)}, '
          f'which is {categories_labels}')

    return coco_json, category_to_id


class Labelme2COCO:

    labels_dir = 'DSEC-Soft/soft_labelme/' + gt_mode + '_' + view + '/'   # Dataset labels directory
    
    # COCO label json output path
    out = out_path
        
    class_id_txt = None   # All class id txt path. class_id_txt will be auto generated.

def save_json_result(out_path, coco_json_data, category_to_id, class_id_txt=None):
    # save json result
    Path(out_path).parent.mkdir(exist_ok=True, parents=True)
    print(f'Saving json to {out_path}')
    json.dump(coco_json_data, open(out_path, 'w'), indent=2)

    if class_id_txt is None:
        category_to_id_path = Path(out_path).with_name('class_with_id.txt')
        print(f'Saving class id txt to {category_to_id_path}')
        with open(category_to_id_path, 'w', encoding='utf-8') as f:
            for k, v in category_to_id.items():
                f.write(f'{v} {k}\n')
    else:
        print('Not Saving new class id txt, user should using '
              f'{class_id_txt} for training config')



# convert to coco json
coco_json_data, category_to_id = parse_labelme_to_coco(image_path_list, Labelme2COCO.labels_dir)

save_json_result(Labelme2COCO.out, coco_json_data, category_to_id)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2/2, 739.7 task/s, elapsed: 0s, ETA:     0s
Total image = 2
Total annotations = 21
Number of categories = 3, which is ['car', 'large_vehicle', 'person']
Saving json to /home/jiangtianbin/mmyolo/DSEC-Soft/soft_ann/soft_ann_COCO_test.json
Saving class id txt to /home/jiangtianbin/mmyolo/DSEC-Soft/soft_ann/class_with_id.txt
